# Fester R-CNN 

selective Search를 사용해 계산했던 Region Proposal 단계를 Neural Network 안으로 포함 

진정한 end-to-end Object Detection 모델 제시

성능 : pascal VOC을 기준으로 78.8%

핵심 idea - Region Proposal network (RPN)

기존 Fest R-CNN 구조를 계승하면서 selective search를 제거하고 RPN을 통해서 RoI를 계산

이를 통해 GPU를 통한 ROI 계산이 가능 and 정확도도 높임


 Object-Detection을 수행하는 알고리즘 

1. feature Map을 먼저 추출한 다음 이를 RPN에 전달하여 ROI 계산 
ROI pooling을 진행한 다음 classificaiotn을 진행하여 Object Detection을 수행한다.


### 1. Region ProPosal Network이 동작하는 object-detection 알고리즘 

1. CNN을 통해 뽑아낸 피쳐 맵을 입력으로 받는다. 이때 피쳐맵의 크기를 H x W x C로 잡는다. (Height, width, channel))
2. 피쳐맵에 3x3 컨볼루션을 256, 512 채널만큼 수행한다. 이때 padding은 1로 설정해주어 h x W 가 보존될 수 있도록 한다.
3. 두번째 피쳐맵을 입력받아서 classidication 과 BoundingBox Regression 예측 값을 계산해 주어야함. 
이때 사용되는 Layer는 Fully Connect Layer가 아니라 1 x 1 convolution을 이용하여 계산하는 Fully convolution network의 특징을 갖는다.
4. 먼저 classidication을 수행하기 위해 1x1 컨볼루션을 2(object여부) x 9(앵커 수) channel 수만큼 수행하며, 그 결과로 H x W x 18 크기의 피쳐 맵을 얻는다. 
각 18개의 channel은 각각 해당 좌표를 앵커로 삼아 k개의 앵커 좌표들에 대한 예측값을 갖고 있다. 이 값들을 적절히 reshape 해준다음 softmax를 적절히 사용하여 오브젝트 일 확률값을 얻는다.


### 2. Multi Task Loss

고정 된 크기의 feature Vector를 구했으니, 각각 Classification, bounding box regression을 적용하여 각각의 Loss를 얻어내고
이를 Back Propagation 하여 전체모델을 학습시키면 된다.

이 때, Classification Loss, bounding box regression을 적절하게 엮어주는 것이 필요한데, 이를 Multi Task Loss 라고 한다.

        L(p, u, t^u, v) = L_cls(p, u) + lambda\[u >= 1\]L_loc(t^u, v)


먼저 입력으로 

softmax

        p = softmax를 통해서 얻어낸 K + 1 (k object + background)개의 확률 값 | p = (p_0, ..., p_k)

        u = 해당 RoI의 Ground Truth 라벨 값

bounding box regression 

k+1개 클래스에 대하여 각각 x, y, w, h 값을 조정하는 tk를 리턴한다. loss function 에는 이들 값 가운데 Ground Truth 라벨에 해당되는 값만 갖고오며 이는 tu에 해당

        t^u_n = (t^u_nx, t^u_ny, t^u_nw, t^u_nh) // 결과중 일부

다시 전체로스로 돌아가면 앞부분은 p, u를 갖고 Classification loss를 구한다. 뒷부분은 bounding box regression 을 통해 얻는다.

        L_cls(p,u) = -logp_u

        L_loc(t^u, v) = sigma(i는 {x, y, w, h}의 부분집합) smooth_L1(t^u_i - v_i)

 smooth_L1(t^u_i - v_i) :
 예측값과 라벨값의 차이를 계산후 smooth_L1이라는 함수를 통과시킨 합을 계산한다.

        smooth_L1(x) = 0.5x^2 (if |x| <1), |x| - 0.5 (otherwise)  




해당 논문은 object detection 테스크를 푸는 end-to-end 모델을 제시하면서 학습 단계를 간소화시키고 정확도와 성능 모두를 향상시켰다는 의의가 있다.

그러나 여전히 region proposal을 selective search로 수행하고, 이는 CPU 연산으로만 수행 가능하다는 한계점이 있습니다. 이 부분을 제외하면 inference에 소요되는 시간이 0.3초 정도로 짧다.

참고 

[1] https://yeomko.tistory.com/15

[2] https://gist.github.com/aisolab/46f73657c7fcbec91004708e01a67ba0